## Create _P.generosa_ tissue-specific lncRNA Expression Matrices

Use lncRNA GTF to ([from 20230502](https://robertslab.github.io/sams-notebook/2023/05/02/lncRNA-Identification-P.generosa-lncRNAs-using-CPC2-and-bedtools.html)) to determine lncRNA expression.

#### Notebook relies on:

- [`stringtie`](https://ccb.jhu.edu/software/stringtie/index.shtml?t=manual)



### List computer specs

In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Thu May  4 11:26:22 PDT 2023
------------



No LSB modules are available.


Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.6 LTS
Release:	18.04
Codename:	bionic

------------
HOSTNAME: 
raven

------------
Computer Specs:

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              48
On-line CPU(s) list: 0-47
Thread(s) per core:  2
Core(s) per socket:  24
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Gold 5220R CPU @ 2.20GHz
Stepping:            7
CPU MHz:             2800.000
CPU max MHz:         4000.0000
CPU min MHz:         1000.0000
BogoMIPS:            4400.00
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            36608K
NUMA node0 CPU(s):   0-47
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdts

### Set variables
- `%env` indicates a bash variable

- without `%env` is Python variable

In [3]:
# Set directories
%env transcriptomes_dir=/home/shared/8TB_HDD_01/sam/data/P_generosa/transcriptomes
%env genomes_dir=/home/shared/8TB_HDD_01/sam/data/P_generosa/genomes
%env analysis_dir=/home/shared/8TB_HDD_01/sam/analyses/20230504-pgen-lncRNA-expression
analysis_dir="20230504-pgen-lncRNA-expression"

# CPU threads
%env threads=40

# Average read length
%env read_length=130

# Input files
%env lncRNA_gtf=20230502-pgen-lncRNA-IDs.gtf
## lncRNA directory URL
## https://gannet.fish.washington.edu/Atumefaciens/20230502-pgen-lncRNA-identification/
%env lncRNA_url=gannet:/volume2/web/Atumefaciens/20230502-pgen-lncRNA-identification

## Tissue-specific sorted BAM files directory URL
## https://gannet.fish.washington.edu/Atumefaciens/20230426-pgen-HISAT2-stringtie-gffcompare-RNAseq/
%env sorted_bams_url=gannet:/volume2/web/Atumefaciens/20230426-pgen-HISAT2-stringtie-gffcompare-RNAseq


# Output file(s)
%env lncRNA_stringtie_gtf=pgen-lncRNA-stringtie.gtf

# Set program locations
%env stringtie=/home/shared/stringtie-2.2.1.Linux_x86_64/stringtie
%env prepDE=/home/shared/stringtie-2.2.1.Linux_x86_64/prepDE.py3

# Line for formatting
%env line=-------------------------------------------------------------------------------------

env: transcriptomes_dir=/home/shared/8TB_HDD_01/sam/data/P_generosa/transcriptomes
env: genomes_dir=/home/shared/8TB_HDD_01/sam/data/P_generosa/genomes
env: analysis_dir=/home/shared/8TB_HDD_01/sam/analyses/20230504-pgen-lncRNA-expression
env: threads=40
env: read_length=130
env: lncRNA_gtf=20230502-pgen-lncRNA-IDs.gtf
env: lncRNA_url=gannet:/volume2/web/Atumefaciens/20230502-pgen-lncRNA-identification
env: sorted_bams_url=gannet:/volume2/web/Atumefaciens/20230426-pgen-HISAT2-stringtie-gffcompare-RNAseq
env: lncRNA_stringtie_gtf=pgen-lncRNA-stringtie.gtf
env: stringtie=/home/shared/stringtie-2.2.1.Linux_x86_64/stringtie
env: prepDE=/home/shared/stringtie-2.2.1.Linux_x86_64/prepDE.py3
env: line=-------------------------------------------------------------------------------------


## Create analysis directories

In [4]:
%%bash

declare -a sample_names_array=(ctenidia gonad heart juvenile larvae)

for sample in "${sample_names_array[@]}"
do
  # Make analysis and data directory, if doesn't exist
  mkdir --parents "${analysis_dir}/${sample}"
done

ls -l "${analysis_dir}"

total 20
drwxrwxr-x 2 sam sam 4096 May  4 11:30 ctenidia
drwxrwxr-x 2 sam sam 4096 May  4 11:30 gonad
drwxrwxr-x 2 sam sam 4096 May  4 11:30 heart
drwxrwxr-x 2 sam sam 4096 May  4 11:30 juvenile
drwxrwxr-x 2 sam sam 4096 May  4 11:30 larvae


## Download lncRNA GTF

In [5]:
%%bash
cd "${transcriptomes_dir}"

rsync "${lncRNA_url}/${lncRNA_gtf}" .


ls -ltrh "${lncRNA_gtf}"

X11 forwarding request failed on channel 0


-rw-rw-r-- 1 sam sam 2.2M May  4 11:30 20230502-pgen-lncRNA-IDs.gtf


### Inspect GTF

In [6]:
%%bash
head "${transcriptomes_dir}/${lncRNA_gtf}"

echo ""
echo "${line}"
echo ""

echo "Number of lines:"
wc -l "${transcriptomes_dir}/"*.gtf

Scaffold_01	StringTie	transcript	656906	657583	.	+	.	transcript_id "MSTRG.38.5"; gene_id "MSTRG.38"; xloc "XLOC_000013"; class_code "u"; tss_id "TSS19";
Scaffold_01	StringTie	transcript	648204	649326	.	+	.	transcript_id "MSTRG.39.1"; gene_id "MSTRG.39"; xloc "XLOC_000014"; class_code "u"; tss_id "TSS20";
Scaffold_01	StringTie	transcript	849165	854552	.	+	.	transcript_id "MSTRG.63.1"; gene_id "MSTRG.63"; xloc "XLOC_000019"; class_code "u"; tss_id "TSS25";
Scaffold_01	StringTie	transcript	852049	854552	.	+	.	transcript_id "MSTRG.63.2"; gene_id "MSTRG.63"; xloc "XLOC_000019"; class_code "u"; tss_id "TSS26";
Scaffold_01	StringTie	transcript	862415	867481	.	+	.	transcript_id "MSTRG.66.1"; gene_id "MSTRG.66"; xloc "XLOC_000020"; class_code "u"; tss_id "TSS27";
Scaffold_01	StringTie	transcript	1824775	1828291	.	+	.	transcript_id "MSTRG.109.1"; gene_id "MSTRG.109"; xloc "XLOC_000040"; class_code "u"; tss_id "TSS56";
Scaffold_01	StringTie	transcript	1966694	1970033	.	+	.	transcript_id "MSTRG.12

## Download tissue-specific BAM files

In [7]:
%%bash
declare -a sample_names_array=(ctenidia gonad heart juvenile larvae)
cd "${transcriptomes_dir}"

for sample in "${sample_names_array[@]}"
do
  rsync -avP "${sorted_bams_url}/${sample}/*.bam" ./"${sample}"/
done


tree -h

X11 forwarding request failed on channel 0


receiving incremental file list

sent 20 bytes  received 73 bytes  186.00 bytes/sec
total size is 6,754,114,204  speedup is 72,624,883.91


X11 forwarding request failed on channel 0


receiving incremental file list

sent 20 bytes  received 69 bytes  178.00 bytes/sec
total size is 6,797,749,344  speedup is 76,379,206.11


X11 forwarding request failed on channel 0


receiving incremental file list

sent 20 bytes  received 70 bytes  180.00 bytes/sec
total size is 12,798,127,359  speedup is 142,201,415.10


X11 forwarding request failed on channel 0


receiving incremental file list

sent 20 bytes  received 72 bytes  61.33 bytes/sec
total size is 40,853,337,318  speedup is 444,058,014.33


X11 forwarding request failed on channel 0


receiving incremental file list

sent 20 bytes  received 71 bytes  182.00 bytes/sec
total size is 8,476,545,854  speedup is 93,148,855.54
.
├── [2.1M]  20230502-pgen-lncRNA-IDs.gtf
├── [4.0K]  ctenidia
│   └── [6.3G]  ctenidia.sorted.bam
├── [4.0K]  gonad
│   └── [6.3G]  gonad.sorted.bam
├── [4.0K]  heart
│   └── [ 12G]  heart.sorted.bam
├── [4.0K]  juvenile
│   └── [ 38G]  juvenile.sorted.bam
└── [4.0K]  larvae
    └── [7.9G]  larvae.sorted.bam

5 directories, 6 files


## Run StingTie to calculate expression values

In [8]:
%%bash
declare -a sample_names_array=(ctenidia gonad heart juvenile larvae)
cd "${transcriptomes_dir}"

time \
for sample in "${sample_names_array[@]}"
do
  "${stringtie}" \
    -G "${lncRNA_gtf}" \
    -e "${sample}/${sample}.sorted.bam" \
    -B \
    -o "${analysis_dir}/${sample}/${sample}-${lncRNA_stringtie_gtf}" \
    -p "${threads}"
done


real	26m57.746s
user	26m26.461s
sys	1m47.458s


### Inspect Ballgown `t_data.ctab` files

In [9]:
%%bash
declare -a sample_names_array=(ctenidia gonad heart juvenile larvae)
cd "${analysis_dir}"

for sample in "${sample_names_array[@]}"
do
  head --verbose "${sample}/t_data.ctab" | column -t
done

==>   ctenidia/t_data.ctab  <==
t_id  chr                   strand  start   end     t_name      num_exons  length  gene_id   gene_name  cov        FPKM
1     Scaffold_01           .       11047   11352   MSTRG.1.1   1          306     MSTRG.1   .          14.054919  12.378725
2     Scaffold_01           .       112427  112635  MSTRG.2.1   1          209     MSTRG.2   .          10.762517  9.478975
3     Scaffold_01           .       144700  145155  MSTRG.3.1   1          456     MSTRG.3   .          4.171104   3.673657
4     Scaffold_01           .       287989  288219  MSTRG.22.1  1          231     MSTRG.22  .          1.844156   1.624221
5     Scaffold_01           -       315962  318697  MSTRG.9.1   1          2736    MSTRG.9   .          0.137427   0.121037
6     Scaffold_01           -       338904  343505  MSTRG.11.1  1          4602    MSTRG.11  .          2.484985   2.188625
7     Scaffold_01           -       340163  341929  MSTRG.12.1  1          1767    MSTRG.12  .         

## Create count matrix

In [10]:
%%bash
cd "${analysis_dir}"

"${prepDE}" \
-l "${read_length}"

ls -ltrh

total 752K
drwxrwxr-x 2 sam sam 4.0K May  4 11:34 ctenidia
drwxrwxr-x 2 sam sam 4.0K May  4 11:37 gonad
drwxrwxr-x 2 sam sam 4.0K May  4 11:41 heart
drwxrwxr-x 2 sam sam 4.0K May  4 11:55 juvenile
drwxrwxr-x 2 sam sam 4.0K May  4 11:58 larvae
-rw-rw-r-- 1 sam sam 409K May  4 11:58 transcript_count_matrix.csv
-rw-rw-r-- 1 sam sam 317K May  4 11:58 gene_count_matrix.csv


### Inspect transcript counts

In [11]:
%%bash
cd "${analysis_dir}"

head transcript_count_matrix.csv | column -t -s ","

echo ""
echo "${line}"
echo ""

wc -l transcript_count_matrix.csv

transcript_id  ctenidia  gonad  heart  juvenile  larvae
MSTRG.1.1      34        16     13     93        6
MSTRG.2.1      18        5      2      9         2
MSTRG.3.1      15        9      48     171       60
MSTRG.22.1     4         24     7      27        22
MSTRG.9.1      3         133    1      1681      245
MSTRG.11.1     88        123    77     144       95
MSTRG.12.1     3         81     12     47        50
MSTRG.25.1     6         47     8      0         1
MSTRG.27.1     4         79     9      12        4

-------------------------------------------------------------------------------------

13607 transcript_count_matrix.csv
